In [50]:
import gradio as gr
import importlib
import re
import os
import pickle
from IPython.display import Markdown
from python_scripts import llm_rag, machine_translation, text_to_speech, whisper_setup, get_audio, utils
import numpy as np
from scipy.io.wavfile import write
# r8_eYbLLc9wPQ1qa4dUK8eXyewFKHvGAck1h9LZl

# Define paths dynamically
PATH = os.getcwd().replace('\\\\', '/')

with open('symptom_list.pkl', 'rb') as f:
    symptom_list = pickle.load(f)

root_path = PATH + '\\Datasets\\MeDAL'
audio_path = PATH + '\\Datasets\\Audio_Files'

In [51]:

importlib.reload(llm_rag)
# importlib.reload(machine_translation)
# importlib.reload(text_to_speech)
# importlib.reload(whisper_setup)
# importlib.reload(get_audio)
# importlib.reload(utils)


<module 'python_scripts.llm_rag' from 'c:\\Users\\Talha\\OneDrive - Higher Education Commission\\Documents\\GitHub\\CS-5302-Project-Group-15\\python_scripts\\llm_rag.py'>

In [52]:
def SMTS(audio):
    try:

        file_path = 'output.wav'
        write(file_path, data = np.array(audio[1], dtype = np.int16), rate = audio[0])
        audio_processed = utils.preprocess_audio(file_path)

        # Step 2: Transcribe Query to English
        whisper_models = ["tiny", "base", "small", "medium", "large"]
        
        transcript = whisper_setup.transcribe_audio(audio_processed, ['tiny'])
        text = (transcript['tiny'][2]).lower()
        print(text)

        # Regular expression pattern to match symptoms containing 'or' any symptoms from the list
        pattern = r'\b(?:' + '|'.join(map(re.escape, symptom_list)) + \
        '|'.join('(?:{}|{})'.format(re.escape(symptom.split(' or ')[0]), re.escape(symptom.split(' or ')[1])) \
                 for symptom in symptom_list if ' or ' in symptom) + r')\b'

        # Extract symptoms from the query
        extracted_symptoms = re.findall(pattern, text, flags = re.IGNORECASE)
        print(extracted_symptoms)

        # Step 3: Feed query into the LLM
        models = { 
        'llama_ours': 'ubaidtariq8/llama2-med-genai', # fine tuned model
        'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
        'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
        'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
        }

        model = llm_rag.DocumentEmbeddingPipeline(model_version = models['mixtral'], chroma_path = root_path)
        model.setup_environment()
        model.prepare_documents(collection_name = "muqeem", joining = True, persistent = True)
        model.embed_and_index()
        query = ('You are a medical doctor. A patient has come to you for desperate need of help. ' +
         'Give as accurate diagnosis as possible tinyd on the symptoms listed. ' +
         ' '.join(extracted_symptoms) + '. Also consider the whole query ' + text + ' ' +
         'Give also suggestions for mitigating the problem.')


        response = model.query_data(query + ' ' + transcript['tiny'][2])
        
        # # Step 4: Translate it back to the user language
        translated_text = machine_translation.translate_text(text = response.response, src_lang = 'en', trg_lang = transcript['tiny'][0])
        print(translated_text)
        # Step 5: Now speak the response in the user's language
        audio_answer_path = audio_path + '/audio.wav'
        text_to_speech.multilingual_text_to_speech(text = translated_text, filepath = audio_answer_path)
        utils.sasti_harkat(audio_answer_path)
        # display(Markdown(f"<b>{translated_text}</b>"))
        return text, translated_text, utils.play_wav(audio_answer_path)
    except Exception as e:
        print("An error occurred:", e)

In [ ]:
demo = gr.Interface(
    SMTS,
    gr.Audio(sources = ["microphone"]),
    ["text", "text", "audio"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 411, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python

 what put the wrong?
[]


Loading files: 100%|██████████| 2/2 [00:00<00:00, 27.86file/s]


Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/71 [00:00<?, ?it/s]